# TSA Chapter 7: Case Study - VECM Impulse Response Functions

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch7/TSA_ch7_case_irf/TSA_ch7_case_irf.ipynb)

Impulse response functions from VECM on FRED GS10 and TB3MS interest rates, 40-step horizon.

In [ ]:
!pip install numpy pandas matplotlib statsmodels pandas_datareader yfinance -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.vector_ar.vecm import VECM
import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD'}
BLUE, RED, GREEN, ORANGE, GRAY, PURPLE = COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange'], COLORS['gray'], COLORS['purple']
plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none', 'savefig.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12, 'axes.labelsize': 10,
    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})
def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

In [ ]:
# Fetch real data from FRED
start, end = '2000-01-01', '2024-12-31'
gs10 = web.DataReader('GS10', 'fred', start, end).dropna()
tb3ms = web.DataReader('TB3MS', 'fred', start, end).dropna()
data = pd.DataFrame({'GS10': gs10['GS10'], 'TB3MS': tb3ms['TB3MS']}).dropna()
print(f'Period: {data.index[0].strftime("%Y-%m")} to {data.index[-1].strftime("%Y-%m")}, n={len(data)}')

# Fit VECM and compute IRF
vecm_model = VECM(data, k_ar_diff=2, coint_rank=1)
vecm_res = vecm_model.fit()
irf = vecm_res.irf(40)
irfs = irf.irfs  # shape: (41, 2, 2) -> [horizon, response_var, shock_var]

horizon = np.arange(irfs.shape[0])

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Panel 1: Shock to Long Rate (GS10)
axes[0].plot(horizon, irfs[:, 0, 0], color=BLUE, lw=2, label='GS10 response')
axes[0].plot(horizon, irfs[:, 1, 0], color=RED, lw=2, label='TB3MS response')
axes[0].axhline(0, color=GRAY, ls='--', lw=0.8)
axes[0].set_title('Shock to 10-Year Rate (GS10)', fontweight='bold')
axes[0].set_xlabel('Months')
axes[0].set_ylabel('Response')
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon=False)

# Panel 2: Shock to Short Rate (TB3MS)
axes[1].plot(horizon, irfs[:, 0, 1], color=BLUE, lw=2, label='GS10 response')
axes[1].plot(horizon, irfs[:, 1, 1], color=RED, lw=2, label='TB3MS response')
axes[1].axhline(0, color=GRAY, ls='--', lw=0.8)
axes[1].set_title('Shock to 3-Month Rate (TB3MS)', fontweight='bold')
axes[1].set_xlabel('Months')
axes[1].set_ylabel('Response')
axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon=False)

plt.tight_layout()
save_chart(fig, 'ch7_case_irf')
plt.show()